In [ ]:
#import theano
#theano.config.openmp=True
#set OMP_NUM_THREADS=4

In [1]:
from __future__ import print_function

import os
import numpy as np
import pandas as pd

# CONSTANTS
BASE_DIR = os.path.dirname('.') # current dir
DB_DIR = os.path.join(BASE_DIR, 'PaHaW', 'PaHaW_public')
DB_INFO = os.path.join(BASE_DIR, 'PaHaW', 'PaHaW_files', 'corpus_PaHaW.csv')
HEADER = ['y', 'x', 't', 'on_surface', 'azimuth', 'altitude', 'pressure']
CHUNK_SIZE=500

def load_db(db_dir, db_info, chunk_size=None, header_names=None):    
    info_data =pd.read_csv(db_info, delimiter=';')
    unique_labels = info_data['Disease']
    
    subjects_idx = 0
    subjects, targets, groups, tasks = [], [], [], []
    for path, dirs, files in os.walk(db_dir):
        if files:
            for file_name in files:
                test_file = os.path.join(path, file_name)
                task_id = int(file_name.split('_')[2])

                # first row contains the number of timesteps
                test_data = pd.read_csv(test_file, delimiter=',', skiprows=1, names=header_names)
                
                # split a sample in several chunks of fixed size
                if chunk_size:
                    n_splits = int(np.ceil(1.0*len(test_data) / chunk_size))
                    chunks = np.array_split(np.array(test_data), n_splits)
                else:
                    # keep the entire sample
                    chunks = [test_data]

                # store each element (entire sample or chunks) with the associated
                # information (control/parkinson, subject_id, task_id) 
                for subject_chunk in chunks:                
                    subjects.append(subject_chunk)
                    targets.append(unique_labels[subjects_idx])
                    groups.append(subjects_idx)
                    tasks.append(task_id)

            subjects_idx += 1
    
    return subjects, np.array(targets), np.array(groups), np.array(tasks) 

In [2]:
# parameters
# directory containing the experiments
#db_dir = DB_DIR
db_dir = os.path.join(BASE_DIR, 'PaHaW', 'PaHaW_public_kf')

# file containing extra information for the experiments
db_info = DB_INFO

# split a sample in several mini-samples (chunks) of fixed size
# chunksize is optional, but disable it for plotting data
#chunk_size = CHUNK_SIZE
chunk_size = None

# header is optional for general use, mandatory for plotting data
#header_names = HEADER
header_names = None
# overwrite this array with new features if neccesary
#header_names = ['f1', 'f2', ...]

# load data
raw_samples, raw_targets, raw_groups, raw_tasks = load_db(db_dir, db_info, 
                                                          chunk_size=chunk_size, 
                                                          header_names=header_names)

# create a copy of the original data (for filtering and postprocessing)
targets = np.copy(raw_targets) 
groups = np.copy(raw_groups)
tasks = np.copy(raw_tasks)

# create an numpy array (copy)
samples = np.array(map(np.array, raw_samples))

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_test(test):
    on_surface = test[test['on_surface'] == 1]
    on_air = test[test['on_surface'] != 1]

    plt.plot(on_surface['x'], on_surface['y'], '.r')
    plt.plot(on_air['x'], on_air['y'], '.b')    

def test_selector(subject_id, task_id):
    # use global variables (they shouldn't be modified)
    global raw_groups, raw_samples, raw_tasks
    index = np.logical_and(raw_groups == subject_id, raw_tasks == task_id)
    index = np.where(index)[0][0]
    return raw_samples[index]

#plot_test(test_selector(1, 1))

In [4]:
# drop list of tasks
# default: just task 1 (spiral)
drop_tasks = [1]
for task_id in drop_tasks:
    keep_tasks = tasks != task_id
    samples, targets, groups, tasks = samples[keep_tasks], targets[keep_tasks], groups[keep_tasks], tasks[keep_tasks]

In [5]:
# create a long vector to compute metrics among all samples
joined_samples = np.concatenate(samples, axis=0)

# compute mean and std by feature
mean_by_feature = np.mean(joined_samples, axis=0)
std_by_feature = np.std(joined_samples, axis=0)
standarize = lambda a :(a-mean_by_feature) / std_by_feature

# compute min and max by feature
min_by_feature = np.min(joined_samples, axis=0)
max_by_feature = np.max(joined_samples, axis=0)
normalize = lambda a :(a-min_by_feature) / np.array(max_by_feature-min_by_feature, dtype=np.float32)

In [6]:
# standarize or normalize the samples
is_standarization = True

if is_standarization:
    samples = np.array(map(standarize, samples))
else:
    samples = np.array(map(normalize, samples))

In [7]:
from keras.preprocessing import sequence

# max number of timesteps
max_timesteps = np.max(map(len, samples))

# pad sequence with zero
print('Pad sequences (samples x time x features)')
padded_samples = sequence.pad_sequences(samples, maxlen=max_timesteps, padding='post', dtype=np.float32)
print('Samples shape:',  padded_samples.shape)

Using Theano backend.


Pad sequences (samples x time x features)
Samples shape: (524L, 7991L, 16L)


In [8]:
# drop selected features 
# default: drop time
#drop_features = [1]
# drop more features
drop_features = [1, 2]

# list of remaining features
keep_features = np.delete(np.arange(padded_samples.shape[2]), drop_features)
final_samples = np.delete(padded_samples, drop_features, axis=2)
n_features = final_samples.shape[2]

print('Final shape:',  final_samples.shape)

Final shape: (524L, 7991L, 14L)


In [ ]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from sklearn.utils import check_random_state

class StratifiedGroupShuffleSplit():
    def __init__(self, test_size=0.2, random_state=None):
        self._random_state = check_random_state(random_state)
        self._test_size = test_size
        self._n_splits=int(1./test_size)
        
    def get_n_splits(self, X, y, g):
        """
        Returns the number of splitting iterations in the cross-validator. Input parameters 
        always ignored, exists for compatibility.
        """
        return self._n_splits
    
    def split(self, X, y, g):
        y_pos = y == 1
        y_pos_idx = np.where(y_pos)[0]
        
        X_p, y_p, g_p = X[y_pos], y[y_pos], g[y_pos]
        group_splitter_p = GroupShuffleSplit(n_splits=self._n_splits, test_size=self._test_size, 
                                             random_state=self._random_state).split(X_p, y_p, g_p)
        y_neg = y == 0
        y_neg_idx = np.where(y_neg)[0]
        
        X_n, y_n, g_n = X[y_neg], y[y_neg], g[y_neg]
        group_splitter_n = GroupShuffleSplit(n_splits=self._n_splits, test_size=self._test_size, 
                                             random_state=self._random_state).split(X_n, y_n, g_n)

        for k in range(self._n_splits):
            train_index_p, test_index_p = next(group_splitter_p)
            train_index_n, test_index_n = next(group_splitter_n)

            train_index = np.hstack([y_pos_idx[train_index_p], y_neg_idx[train_index_n]])        
            test_index  = np.hstack([y_pos_idx[test_index_p], y_neg_idx[test_index_n]])

            self._random_state.shuffle(train_index)            
            self._random_state.shuffle(test_index)
            
            yield (train_index, test_index)

def stratified_group_train_test_split(X, y, g, test_size=0.2, random_state=None, output_groups=False):
    splitter = StratifiedGroupShuffleSplit(test_size=test_size, random_state=random_state)
    
    for train, test in splitter.split(X, y, g):
        # exit after 1 iteration

        if output_groups:
            return X[train], X[test], y[train], y[test], g[train], g[test]
        else:
            return X[train], X[test], y[train], y[test]

In [ ]:
# random number generator (for experiments reproduction)
seed = 42
rs = check_random_state(seed)

# Split dataset in train and test partitions
test_size = 1./5
X_train, X_test, y_train, y_test, g_train, g_test = stratified_group_train_test_split(final_samples, targets, groups, test_size, random_state=rs, output_groups=True)

In [ ]:
# Split dataset for k-fold validation
val_test_size=1./5
k_fold_splitter = StratifiedGroupShuffleSplit(val_test_size, random_state=rs)

In [ ]:
print('Loading data...')
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Masking
from keras.layers import LSTM, SimpleRNN, GRU
from keras.optimizers import Adam

def create_model(mtype='lstm', max_timesteps=None, n_features=None, units=None, dropout_rate=0.0, learning_rate=0.01):
    model = Sequential()
    # skip zero padding
    model.add(Masking(mask_value=0., input_shape=(max_timesteps, n_features)))

    # weights initialization may be important
    print('Choosen model: %s' % (mtype,))

    weights_init ='glorot_uniform'
    internal_activation = 'tanh'
    if mtype == 'lstm':
        model.add(LSTM(units, dropout_W=0.2, dropout_U=0.2, init=weights_init, activation=internal_activation))
    elif mtype == 'simple':
        model.add(SimpleRNN(units, dropout_W=0.2, dropout_U=0.2, init=weights_init, activation=internal_activation))
    elif mtype == 'gru':
        model.add(GRU(units, dropout_W=0.2, dropout_U=0.2, init=weights_init, activation=internal_activation))        
    else:
        print('Model %s not implementd: %s' % (mtype,))       
        return None
        
    # dropout
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))

    # basic activation functions can be relu, tanh or sigmoid
    model.add(Activation('sigmoid'))

    # try using different optimizers and different optimizer configs
    optimizer = Adam(lr=learning_rate, decay=1e-6)    
    
    # REVIEW: is binary_crossentropy the correct loss function ? 
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

In [5]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

RuntimeError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [
  {'units': [16, 32, 64, 128], 'dropout_rate': [0, 0.25, 0.5], 'learning_rate': [0.005], 'n_features': [n_features], 'max_timesteps': [max_timesteps],},
]

batch_size = 64
nb_epoch = 10

scikit_model = KerasClassifier(build_fn=create_model, batch_size=batch_size, nb_epoch=nb_epoch, verbose=2)

grid = GridSearchCV(estimator=scikit_model, param_grid=param_grid, 
                    n_jobs=1, iid=False, cv=k_fold_splitter, verbose=2)
grid_result = grid.fit(X_train, y_train, g_train)

In [ ]:
grid_result_df = pd.DataFrame(grid_result.cv_results_)
grid_result_file = os.path.join(BASE_DIR, 'grid_results.csv')
grid_result_df.to_csv(grid_result_file)

In [ ]:
# best parameters obtained from k-fold validation
lstm_units = 32
dropout_rate=0.2
learning_rate=0.01

print('Build model...')
model = create_model(mtype='lstm', max_timesteps=max_timesteps, n_features=n_features, units=lstm_units, dropout_rate=dropout_rate, learning_rate=learning_rate)
model.summary()

In [ ]:
import os
from keras.models import load_model

def run_action(action, **kwargs):   
    if action == 'reset':
        # reset internal state if needed
        model = kwargs['model']
        model.reset_states()
        return model
    
    elif action == 'load':
        # load internal state from file
        if os.path.exists(kwargs['path']):
            return load_model(kwargs['path'])
        else:
            print('Error: File %s not found.' % (kwargs['path'],))            

    elif action == 'save':
        # save internal state
        model = kwargs['model']
        model.save(kwargs['path'])
        return model
    else:
        print('Error: Action %s not implemented' % (action,))

# Examples
#fname = os.path.join(BASE_DIR, 'lstm_' + str(lstm_units) + '.h5')
#model = run_action('load', path=fname)
#run_action('save', path=fname, model=model)
#run_action('reset', model=model)

In [ ]:
import math
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
from keras import callbacks
from keras.models import load_model

batch_size = 64
nb_epoch = 20

cbks = [ 
        callbacks.History(),     
#       callbacks.ModelCheckpoint(filepath=fname, monitor='val_loss', save_best_only=True),
#       callbacks.EarlyStopping(monitor='val_loss', patience=5),
#       callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
#       callbacks.LearningRateScheduler(step_decay),
       ]

print('Train...')

history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=2, 
#                    validation_data=(X_test, y_test), 
                    callbacks=cbks)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, accuracy_score

# create scikit wrapper
scikit_model = KerasClassifier(build_fn=create_model, verbose=0)
# workaround
scikit_model.model = model

In [ ]:
train_preds = scikit_model.predict(X_train)
acc = accuracy_score(y_train, train_preds)
print('Train accuracy = {:.4f}'.format(acc))
print(classification_report(y_train, train_preds, target_names=['control', 'parkinson']))

In [ ]:
test_preds = scikit_model.predict(X_test)

acc = accuracy_score(y_test, test_preds)
print('Test accuracy = {:.4f}'.format(acc))
print(classification_report(y_test, test_preds, target_names=['control', 'parkinson']))

In [ ]:
import smtplib
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText
from email.MIMEBase import MIMEBase
from email import encoders
 
fromaddr = "juanluisrosaramos@gmail.com"
toaddr = "juanluisrosaramos@gmail.com"
 
msg = MIMEMultipart()
 
msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = "CI RESULTS!"
 
body = "TEXT YOU WANT TO SEND"
 
msg.attach(MIMEText(body, 'plain'))
 
filename = os.path.join(BASE_DIR, 'lstm_' + str(32) + '.h5')
attachment = open(filename, "rb")
 
part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
 
msg.attach(part)
 
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "dsrtrtyu")
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()
attachment.close()